In [367]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [368]:
import pandas as pd
import numpy as np

In [369]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [370]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_92640/1060631013.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


In [371]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )

sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])
                            # sum_distrib_df['from_name'] + ' - ' \
                            #     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(distrib_df)

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_92640/2744235436.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()


In [372]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
df = sum_distrib_df.merge(tvl, on='join_key',how='outer')
# display(df[df['join_key'].str.contains('velodr')])

In [373]:

mask = (df['from_name'] == 'Velodrome') & (df['program_name'].str.contains('Velodrome'))

programs = df.loc[mask, 'program_name'].unique()

for program in programs:
    # Find the indices of rows for this program where from_name is not Velodrome
    indices = df.loc[(df['program_name'] == program) & (df['from_name'] != 'Velodrome')].index
    
    # If there are any such rows, set their from_name to be the from_name of the first Velodrome row for this program
    if len(indices) > 0:
        velodrome_index = df.loc[(df['program_name'] == program) & (df['from_name'] == 'Velodrome')].index[0]
        non_velodrome_index = indices[0]
        df.loc[velodrome_index, 'from_name'] = df.loc[non_velodrome_index, 'from_name']
    # Otherwise, set the from_name of the first Velodrome row for this program to be "Unknown"
    else:
        velodrome_index = df.loc[(df['program_name'] == program) & (df['from_name'] == 'Velodrome')].index[0]
        df.loc[velodrome_index, 'from_name'] = 'Unknown'

display(df[df['join_key'].str.contains('poolt')])
print(df.columns)

,from_name,program_map,op_deployed,join_key,date,include_in_summary,top_level_name,program_name,app_name,num_op,...,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention
57,PoolTogether,,842488.0,pooltogether,2023-03-10,1.0,Pooltogether,Pooltogether,Pooltogether,450000.0,...,During Program,Partner Fund,2022-07-22,NaN,3.695229e+07,3.695229e+07,3.695232e+07,3.695232e+07,NaN,NaN


Index(['from_name', 'program_map', 'op_deployed', 'join_key', 'date',
       'include_in_summary', 'top_level_name', 'program_name', 'app_name',
       'num_op', 'num_op_override', 'period', 'op_source', 'start_date',
       'end_date', 'cumul_net_dollar_flow_at_program_end',
       'cumul_net_dollar_flow',
       'cumul_last_price_net_dollar_flow_at_program_end',
       'cumul_last_price_net_dollar_flow', 'flows_retention',
       'last_price_net_dollar_flows_retention'],
      dtype='object')


In [374]:
#Create the aggregate app name field
df['agg_app_name'] = df['from_name'].combine_first(df['app_name'])
df = df.fillna(0) #Fill NA with 0
display(df)

,from_name,program_map,op_deployed,join_key,date,include_in_summary,top_level_name,program_name,app_name,num_op,...,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention,agg_app_name
0,1inch,,300000.0,1inch,0,0.0,0,0,0,0.000000e+00,...,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,1inch
1,Aelin,,900002.0,aelin,2022-10-14,1.0,Aelin,Aelin,Aelin,9.000000e+05,...,Gov Fund - Phase 0,2022-09-12,2022-09-14,-2.827652e+03,-2.778652e+03,-2.826693e+03,-2.777707e+03,-0.982671,-0.982670,Aelin
2,Alchemix,,20386.0,alchemix,0,0.0,0,0,0,0.000000e+00,...,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,Alchemix
3,Alchemix,alETH/WETH: Velodrome,10000.0,aleth/weth: velodrome,2023-03-10,1.0,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,Alchemix,0.000000e+00,...,Gov Fund - Season 1,2023-01-08,0,3.541272e+06,3.541272e+06,3.533613e+06,3.533613e+06,0.000000,0.000000,Alchemix
4,Alchemix,alETH/WETH: Velodrome,3366.0,aleth/weth: velodrome,2023-03-10,1.0,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,Alchemix,0.000000e+00,...,Gov Fund - Season 1,2023-01-08,0,3.541272e+06,3.541272e+06,3.533613e+06,3.533613e+06,0.000000,0.000000,Alchemix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,0,0,0.0,uniswap lm - phase 1,2022-12-21,1.0,Uniswap LM - Phase 1,Arrakis Finance - Uniswap LM - Phase 1,Arrakis Finance,1.666667e+04,...,Gov Fund - Phase 0,2022-10-26,2022-11-21,1.982708e+06,7.457713e+05,2.195283e+06,9.202369e+05,0.376138,0.419188,Arrakis Finance
109,0,0,0.0,snx bridge: hop,2023-03-10,1.0,SNX Bridge: Hop,Synthetix - SNX Bridge: Hop,Synthetix,0.000000e+00,...,Gov Fund - Phase 0,2022-10-25,0,1.514966e+06,1.514966e+06,1.481664e+06,1.481664e+06,0.000000,0.000000,Synthetix
110,0,0,0.0,xtoken,2023-03-10,1.0,Xtoken,Xtoken,Xtoken,9.000000e+05,...,Gov Fund - Season 1,2022-09-19,0,1.359404e+06,1.359404e+06,1.590989e+06,1.590989e+06,0.000000,0.000000,Xtoken
111,0,0,0.0,aave - liquidity mining,2022-12-04,1.0,Aave - Liquidity Mining,Aave - Aave - Liquidity Mining,Aave,5.000000e+06,...,Partner Fund,2022-08-04,2022-11-04,3.450056e+08,7.849483e+07,3.419694e+08,7.732963e+07,0.227518,0.226130,Aave


In [375]:
# Now union back again

select_cols = ['agg_app_name','top_level_name', 'program_name',\
               'num_op_override', 'period','op_source', 'start_date','end_date'
               ,'op_deployed', 
               'cumul_net_dollar_flow_at_program_end',
                'cumul_net_dollar_flow',
                'cumul_last_price_net_dollar_flow_at_program_end',
                'cumul_last_price_net_dollar_flow'
                ]
group_cols = select_cols[:8] #group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

['agg_app_name', 'top_level_name', 'program_name', 'num_op_override', 'period', 'op_source', 'start_date', 'end_date']


In [376]:
#Get the rank by start_date of each program

# replace 0s with '1900-01-01'
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == 0, '1900-01-01', sum_distrib_df['start_date'])
# create a new column 'program_rank' based on the 'start_date' column
sum_distrib_df['program_rank'] = sum_distrib_df.groupby('agg_app_name')['start_date'].rank(ascending=True)
# replace '1900-01-01' with 0s
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == '1900-01-01', 0, sum_distrib_df['start_date'])

# subtract all overridden values fromthe amount I have deployed

# create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
sum_distrib_df['cumulative_num_op_override'] = sum_distrib_df.groupby('agg_app_name')['num_op_override'].cumsum()
# create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
sum_distrib_df['op_deployed_net_override'] = sum_distrib_df['op_deployed'] - sum_distrib_df.groupby('agg_app_name')['cumulative_num_op_override'].shift(1).fillna(0)
# drop the 'cumulative_num_op_override' column
sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

In [377]:
# display(sum_distrib_df)
display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('Velodr')])

,agg_app_name,top_level_name,program_name,num_op_override,period,op_source,start_date,end_date,op_deployed,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,program_rank,op_deployed_net_override
85,Velodrome,0,0,0.0,0,0,0,0,4400739.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,4400739.0
86,Velodrome,Velodrome #1 *,Velodrome - Velodrome #1 *,3000000.0,Post-Program,Partner Fund,2022-07-13,2022-11-17,0.0,8.746736e+07,4.748105e+07,8.746736e+07,4.748105e+07,2.0,0.0
87,Velodrome,Velodrome #2 (Tour de OP)*,Velodrome - Velodrome #2 (Tour de OP)*,0.0,During Program,Gov Fund - Season 2,2022-11-24,0,0.0,2.295112e+08,2.295112e+08,2.295112e+08,2.295112e+08,3.0,-3000000.0
